In [4]:
import numpy as np
aa = np.array([1,4,6,9,23])

result = np.where(aa == np.amax(aa))
print(result, np.amax(aa))

(array([4], dtype=int64),) 23


In [11]:
import numpy as np
aa = np.array([1,4,6,4,23])
np.where(aa == 5)

(array([], dtype=int64),)

In [3]:
from Model.InteractionNode import InteractionNode

In [17]:
print("===")
print("Product={}, PriceLevel={}, Bought={}, #Units={}".format(2,3,"Si",45))


l21 = InteractionNode(3, 4, 0, 0, [])
l22 = InteractionNode(2, 1, 0, 0, [])
l1 = InteractionNode(1, 2, 1, 34, [l21,l22])

l1.printInteractions()

===
Product=2, PriceLevel=3, Bought=Si, #Units=45

USER INTERACTIONS: 
╚═ Product=1, PriceLevel=2, Bought=Yes, #Units=34
  ╚═ Product=3, PriceLevel=4, Bought=No, #Units=0
  ╚═ Product=2, PriceLevel=1, Bought=No, #Units=0


In [19]:
import numpy as np
np.random.uniform(low=0.0, high=1.0, size=None)

0.27985094518957787

In [ ]:
import pprint
import sys
import unittest
from Model.GraphProbabilities import GraphProbabilities
from Model.Product import Product
from Model.UserClass import UserClass
from Model.constants import PROBABILITY_MATRIX, SECONDARY_PRODUCTS

id = 0
conversionRate = [
    [1, 0.3, 0.9, 0.2],
    [1, .2, .56, .12],
    [1, .34, .57, .88],
    [1, .2, .56, .12],
    [1, 0.9, 0.17, 0.96]
]

PROBABILITY_MATRIX_3 = [[0, 1, 1, 0, 0],
                      [0.8, 0, 0, 0.9, 0],
                      [0, 1, 0, 0.7, 0],
                      [0.9, 0, 0, 0, 0],
                      [0, 0, 0, 0, 0]]
clickProbability = GraphProbabilities(PROBABILITY_MATRIX_3)
alphas = [1,0,0,0,0]

n_bought_mean = 2
n_bought_variance = 1
n_user_mean = 10
n_user_variance = 1

productList = [Product(int(key), SECONDARY_PRODUCTS[key]) for key in SECONDARY_PRODUCTS]

Lambda = 1

userClass = UserClass(id=id, conversionRate=conversionRate, clickProbability=clickProbability, alphas=alphas,
                        Lambda=Lambda, n_bought_variance=n_bought_variance, n_bought_mean=n_bought_mean,
                        n_user_mean=n_user_mean, n_user_variance=n_user_variance, productList=productList,
                        features_generator=[{"name":"Over 18","probability":0.6},{"name":"Male","probability":0.9}],
                        debug=False)

# pp = pprint.PrettyPrinter(indent=4)
# pp.pprint(userClass.__dict__)

currentPrice = [0, 0, 0, 0, 0]
userClass.setCurrentPrice(currentPrice)

userInteractions = userClass.generateEpisode()
userInteractions.printInteractions()

In [ ]:
def yummy(time, env):
    print("Time: {}".format(time))
    print("Env: {}".format(env.Lambda))
from Environment import Environment
e = Environment(0.3, [userClass])
e.addTimeListener(2, yummy)
e.addTimeListener(2, yummy)
e.round()